# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kapuskasing,49.4169,-82.4331,3.05,93,100,2.57,CA,2024-11-03 18:00:51
1,1,wailua homesteads,22.0669,-159.3780,27.70,80,95,2.24,US,2024-11-03 18:00:52
2,2,port-aux-francais,-49.3500,70.2167,4.82,87,55,12.96,TF,2024-11-03 18:00:53
3,3,ushuaia,-54.8000,-68.3000,6.81,81,0,2.57,AR,2024-11-03 18:00:54
4,4,freetown,8.4840,-13.2299,26.11,83,100,2.42,SL,2024-11-03 18:00:55


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

import pandas as pd
import hvplot.pandas

# Assuming city_data_df is already defined and contains the necessary data
# Ensure that the DataFrame has columns for 'Lat', 'Lng', and 'Humidity'

# Create the map with color coding based on city names
city_map = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    tiles='OSM', 
    size='Humidity',  # Size of the points based on humidity
    color='City',  # Color coding based on city names
    frame_width=800, 
    frame_height=600,
    title='City Humidity Map',
    cmap='Category10'  # You can choose a categorical color map
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Define your ideal weather conditions
ideal_temperature_min = 20  # Minimum ideal temperature in Fahrenheit
ideal_temperature_max = 100  # Maximum ideal temperature in Fahrenheit
max_humidity = 80            # Maximum ideal humidity percentage
max_cloudiness = 50          # Maximum ideal cloudiness percentage
max_wind_speed = 15          # Maximum ideal wind speed in mph

# Narrow down cities that fit criteria
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] >= ideal_temperature_min) &
    (city_data_df['Max Temp'] <= ideal_temperature_max) &
    (city_data_df['Humidity'] <= max_humidity) &
    (city_data_df['Cloudiness'] <= max_cloudiness) &
    (city_data_df['Wind Speed'] <= max_wind_speed)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Select the relevant columns to match the intended output format
ideal_weather_df = ideal_weather_df[['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Country', 'Date']]

# Limit to 20 rows
ideal_weather_df = ideal_weather_df.head(20)

# Display sample data
if not ideal_weather_df.empty:
    print(ideal_weather_df)  # Display the filtered DataFrame
else:
    print("No cities meet the ideal weather conditions.")

     City_ID                      City      Lat       Lng  Max Temp  Humidity  \
16        16                    hasaki  35.7333  140.8333     21.08        57   
28        28              saint-pierre -21.3393   55.4781     22.82        68   
37        37                      ghat  24.9647   10.1728     22.22        33   
41        41               broken hill -31.9500  141.4333     23.91        30   
50        50                    ormara  25.2088   64.6357     26.98        73   
58        58                  san juan -31.5375  -68.5364     24.76        46   
63        63                  coruripe -10.1256  -36.1756     24.25        73   
71        71                  sultanah  24.4926   39.5857     25.25        53   
72        72              puerto ayora  -0.7393  -90.3518     20.07        75   
81        81                 thinadhoo   0.5333   72.9333     27.76        70   
84        84                san andres  12.5847  -81.7006     29.01        79   
85        85                

### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Assuming ideal_weather_df is already defined and contains the relevant columns
# Create a DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ""

# Optionally, if you want to keep the same number of rows as in ideal_weather_df (which should be 20)
# Ensure that hotel_df contains the same 20 cities
hotel_df = hotel_df.head(20)  # This is optional if ideal_weather_df already has 20 rows

# Display the entire hotel_df to verify the output
print(hotel_df)  # Display the DataFrame

                         City Country      Lat       Lng  Humidity Hotel Name
16                     hasaki      JP  35.7333  140.8333        57           
28               saint-pierre      RE -21.3393   55.4781        68           
37                       ghat      LY  24.9647   10.1728        33           
41                broken hill      AU -31.9500  141.4333        30           
50                     ormara      PK  25.2088   64.6357        73           
58                   san juan      AR -31.5375  -68.5364        46           
63                   coruripe      BR -10.1256  -36.1756        73           
71                   sultanah      SA  24.4926   39.5857        53           
72               puerto ayora      EC  -0.7393  -90.3518        75           
81                  thinadhoo      MV   0.5333   72.9333        70           
84                 san andres      CO  12.5847  -81.7006        79           
85                   kingston      JM  17.9970  -76.7936        

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000  # Radius in meters

# Define the parameters for the API request
params = {
    "categories": "hotel",  # Specify the category for hotels
    "apiKey": geoapify_key,  # Your Geoapify API key
    "limit": 1,  # Limit to 1 result (the first hotel found)
    "bias": "proximity"  # Prioritize results based on proximity to the coordinates
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"  # Format for Geoapify API
    params["bias"] = f"proximity:{longitude},{latitude}"  # Bias towards the specified coordinates

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hasaki - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
ghat - nearest hotel: No hotel found
broken hill - nearest hotel: No hotel found
ormara - nearest hotel: No hotel found
san juan - nearest hotel: No hotel found
coruripe - nearest hotel: No hotel found
sultanah - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
thinadhoo - nearest hotel: No hotel found
san andres - nearest hotel: No hotel found
kingston - nearest hotel: No hotel found
sao jose da coroa grande - nearest hotel: No hotel found
saipan - nearest hotel: No hotel found
manokwari - nearest hotel: No hotel found
ebaye - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
ciudad lazaro cardenas - nearest hotel: No hotel found
natal - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
16,hasaki,JP,35.7333,140.8333,57,No hotel found
28,saint-pierre,RE,-21.3393,55.4781,68,No hotel found
37,ghat,LY,24.9647,10.1728,33,No hotel found
41,broken hill,AU,-31.9500,141.4333,30,No hotel found
50,ormara,PK,25.2088,64.6357,73,No hotel found
58,san juan,AR,-31.5375,-68.5364,46,No hotel found
63,coruripe,BR,-10.1256,-36.1756,73,No hotel found
71,sultanah,SA,24.4926,39.5857,53,No hotel found
72,puerto ayora,EC,-0.7393,-90.3518,75,No hotel found
81,thinadhoo,MV,0.5333,72.9333,70,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

import pandas as pd
import hvplot.pandas

# Assuming hotel_df is already defined and contains the relevant columns including 'Hotel Name' and 'Country'

# Configure the map plot
city_map = hotel_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    tiles='OSM', 
    size='Humidity',  # Size of the points based on humidity
    color='City',  # Color coding based on city names
    frame_width=800, 
    frame_height=600,
    title='City Humidity Map',
    cmap='Category10',  # Use a categorical color map
    hover_cols=['City', 'Country', 'Hotel Name']  # Add hover information
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)